In [24]:
import subprocess
from pathlib import Path

materials = "h-CrO2"
filename = f"ptest/{materials}_1.lmp"
Path(filename).unlink(missing_ok=True)


atomsk_command = f"atomsk tribo_2D/cif/{materials}.cif -orthogonal-cell -duplicate 2 1 1 -ow {filename} "
subprocess.run(atomsk_command, shell=True, check=True)

 ___________________________________________________
|              ___________                          |
|     o---o    A T O M S K                          |
|    o---o|    Version Beta 0.13.1                  |
|    |   |o    (C) 2010 Pierre Hirel                |
|    o---o     https://atomsk.univ-lille.fr         |
|___________________________________________________|
*** Working out of office hours? You should sleep sometimes. :-)
>>> Opening the input file: tribo_2D/cif/h-CrO2.cif
..> Applying symmetry operations...
..> Input file was read successfully (3 atoms).
>>> Converting system into an orthorhombic cell...
..> A suitable cell was found, now filling it with atoms...
    (estimated new number of atoms: 7)
..> Cell is now orthorhombic (6 atoms).
>>> Duplicating the system: 2 x 1 x 1
..> System was successfully duplicated (12 atoms).
>>> Writing output file(s) (12 atoms):
..> Successfully wrote LAMMPS data file: ptest/h-CrO2_1.lmp
\o/ Program terminated successfully!
    Tot

CompletedProcess(args='atomsk tribo_2D/cif/h-CrO2.cif -orthogonal-cell -duplicate 2 1 1 -ow ptest/h-CrO2_1.lmp ', returncode=0)

In [ ]:
import os
import subprocess
import numpy as np
import re
from pathlib import Path
from ase.io import read

x=100
y=100

# Now, you can import tribo_2D
from tribo_2D import tools, settings
with open("material_list.txt", "r") as f:
    materials = [line.strip() for line in f]

materials = ["h-CrO2"]

# atomsk_command = f"atomsk Program/tribo_2D/cif/{material}.cif -duplicate 2 2 1 -orthogonal-cell a.lmp"
for mat in materials:
    filename = f"ptest/{mat}_1.lmp"
    
    pot = tools.count_elemtypes(f"tribo_2D/Potentials/sw_lammps/{mat}.sw")
    cif = tools.cifread(f"tribo_2D/cif/{mat}.cif")

    multiples = {}

    for element, cif_count in cif["elem_comp"].items():
        potential_count = pot.get(element, 0)  # Get count from potential file or default to 0
        if cif_count > 0 and potential_count != 1:  # Avoid division by zero
            multiples[element] = potential_count / cif_count
        else:
            multiples[element] = None  # If the CIF count is zero (which shouldn't happen), mark as None

    first_multiple = next(iter(multiples.values()))  # Get the first multiple

    for multiple in multiples.values():
        if multiple != first_multiple:  
            if multiple == None and first_multiple == 1 or multiple == 1 and first_multiple== None:
                first_multiple=1
            else:
                raise ValueError("multiples must be the same")

    typecount = 0
    for atomcount in pot.values():
        typecount += atomcount
 
    Path(filename).unlink(missing_ok=True)

    if first_multiple is None:
        atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -ow {filename} -v 0"

    elif isinstance(first_multiple, float) and not first_multiple.is_integer():
        raise ValueError('potential file or cif file is not formatted properly')
    
    else:
        if first_multiple ==1:
            atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -ow {filename} -v 0"
        else:
            atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -orthogonal-cell -ow {filename} -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            _ = tools.num_atoms_lmp(filename)

            atomsk_command = f"echo n | atomsk {filename} -orthogonal-cell -ow lmp -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            atoms = read(filename, format="lammps-data")
            # Get number of atoms
            natoms = len(atoms)
            print(natoms)
            if typecount % natoms == 0:
                for i in range(int(np.sqrt(typecount/natoms))+1, 0, -1):
                    if typecount/natoms % i == 0:
                        a = int(i)
                        b = int(typecount/natoms / i)
                        break

            atomsk_command = f"echo n | atomsk {filename} -duplicate {a} {b} 1 -ow lmp -v 0"
            print(a,b)
    
    print(atomsk_command)
    subprocess.run(atomsk_command, shell=True, check=True)

    elem2D = tools.num_atoms_lmp(filename)

    if first_multiple == None or first_multiple == 1:
        elem2D = tools.num_atoms_lmp(filename)
        atomsk_command = f"atomsk {filename} -orthogonal-cell -ow lmp"
        subprocess.run(atomsk_command, shell=True, check=True)


dim = tools.get_model_dimensions(filename)
duplicate_a = round(x / dim['xhi'])
duplicate_b = round(y / dim['yhi'])

atomsk_command = f"atomsk {filename} -duplicate {duplicate_a} {duplicate_b} 1 -ow lmp -v 0"
subprocess.run(atomsk_command, shell=True, check=True)

    # atomsk_command = f"atomsk {filename} -duplicate 2 2 1 -ow lmp"
    # subprocess.run(atomsk_command, shell=True, check=True)

AttributeError: module 'tribo_2D.tools' has no attribute 'num_atoms_lmp'